<a href="https://colab.research.google.com/github/JozefSL/2022/blob/main/colab/NDMWD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import xlrd
from xlrd import XLRDError
import csv
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from google.colab import drive
import os
import sqlite3

In [3]:
def PullNDData():
   
    # create list of all yyyy-mm for data download.  The first free file available in xcel is from 2015-04-10
    # Access the ND DMR data @ https://www.dmr.nd.gov/oilgas/mprindex.asp
    # select the month of the first file to scrape:
    ymList = pd.date_range('2021-04-10',datetime.today()- relativedelta(months=2), freq='MS').strftime("%Y_%m").tolist()  # with month in numeric format
   
    website = "https://www.dmr.nd.gov/oilgas/mpr/"

    NDMWD = []  #list()  # North Dakota Monthly Well Data
    for month in ymList:
        try:
            target_url = website + month + '.xlsx'
            print(target_url)
            r = requests.get(target_url)
            wb = pd.read_excel(xlrd.open_workbook(file_contents=r.content))
            NDMWD.append(wb)
        except: #finds exceptions when pulling data from website, logs and displays errors to screen. Will add to file.
            if(RuntimeError):
                print(RuntimeError)
            elif(XLRDError):
                print(XLRDError)
            print("error month: ", month)
            pass

    NDMWD = pd.concat(NDMWD)
    NDMWD.rename(columns={"API_WELLNO": "APINumber"},inplace=True)
    NDMWD['APINumber'] = NDMWD['APINumber'].astype(str) #sets APINumber column as a string/character

    
    print('ND well data query executed on', datetime.now().strftime("%A, %B %d, %Y at %I:%M%p"),'(GMT')
    return NDMWD
NDMWD = PullNDData()

https://www.dmr.nd.gov/oilgas/mpr/2021_05.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2021_06.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2021_07.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2021_08.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2021_09.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2021_10.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2021_11.xlsx
<class 'RuntimeError'>
error month:  2021_11
ND well data query executed on Wednesday, January 05, 2022 at 10:20PM (GMT


In [8]:
NDMWD.head()

,ReportDate,APINumber,FileNo,Company,WellName,Quarter,Section,Township,Range,County,FieldName,Pool,Oil,Wtr,Days,Runs,Gas,GasSold,Flared,Lat,Long
0,2021-05-01,33053050000000,25644,EQUINOR ENERGY LP,BILL 14-23 6H,NWNE,14,151,101,MCK,ALEXANDER,BAKKEN,1645.0,3577.0,31.0,1825,6686.0,6562,0.0,47.904582,-103.569419
1,2021-05-01,33053039010000,22023,EQUINOR ENERGY LP,BILL 14-23 1H,SWSW,11,151,101,MCK,ALEXANDER,BAKKEN,753.0,822.0,31.0,688,5060.0,3875,1061.0,47.907508,-103.580354
2,2021-05-01,33053038990000,22021,EQUINOR ENERGY LP,BILL 14-23 2TFH,SWSW,11,151,101,MCK,ALEXANDER,BAKKEN,941.0,2366.0,31.0,1004,839.0,615,100.0,47.907450,-103.580411
3,2021-05-01,33053048330000,25091,EQUINOR ENERGY LP,BILL 14-23 3H,NWNE,14,151,101,MCK,ALEXANDER,BAKKEN,533.0,3583.0,31.0,501,488.0,364,0.0,47.904582,-103.569664
4,2021-05-01,33053050010000,25645,EQUINOR ENERGY LP,BILL 14-23 4TFH,NWNE,14,151,101,MCK,ALEXANDER,BAKKEN,854.0,7610.0,31.0,802,998.0,874,0.0,47.904582,-103.569541


In [5]:
firstProdMonth = NDMWD[['ReportDate','APINumber','Oil','Gas','Days']]
firstProdMonth = firstProdMonth.sort_values(['APINumber','ReportDate']).drop_duplicates('APINumber', keep='first')
firstProdMonth.rename({'ReportDate': 'firstProdMonth'}, axis=1, inplace=True)

In [6]:
firstProdMonth['oP0'] = firstProdMonth['Oil']/firstProdMonth['Days']
firstProdMonth['gP0'] = firstProdMonth['Gas']/firstProdMonth['Days']
firstProdMonth = firstProdMonth.drop(['Oil','Gas','Days'],axis=1)
firstProdMonth = firstProdMonth.dropna().reset_index(drop=True)

In [7]:
#firstProdMonth.tail()
firstProdMonth.info()
#firstProdMonth[firstProdMonth['APINumber']== '33007000400000']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16622 entries, 0 to 16621
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   firstProdMonth  16622 non-null  datetime64[ns]
 1   APINumber       16622 non-null  object        
 2   oP0             16622 non-null  float64       
 3   gP0             16622 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 519.6+ KB


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/FF_data')
print("connected to sqlite3 verion:", sqlite3.version)
firstProdMonth.to_sql('firstProdMonth', conn, index=False, if_exists='replace')
conn.close()
print('Updated firstProdMonth table on', datetime.now().strftime("%A, %B %d, %Y at %I:%M%p"),'(GMT)' '\nconnection closed')

connected to sqlite3 verion: 2.6.0
Updated firstProdMonth table on Tuesday, December 21, 2021 at 02:15AM (GMT)
connection closed


In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/FracFocus/FF_data')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

print(cursor.fetchall())
conn.close()

[('RegistryUpload',), ('FracFocusRegistry',), ('firstProdMonth',)]


In [ ]:
os.listdir()

['.config', 'drive', 'sample_data']

In [ ]:
!ls

drive  sample_data
